## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**2072. The Winner University (Easy)**

**Table: NewYork**

| Column Name | Type |
|-------------|------|
| student_id  | int  |
| score       | int  |

In SQL, student_id is the primary key for this table.
Each row contains information about the score of one student from New York University in an exam.
 
**Table: California**

| Column Name | Type |
|-------------|------|
| student_id  | int  |
| score       | int  |

In SQL, student_id is the primary key for this table.
Each row contains information about the score of one student from California University in an exam.
 
There is a competition between New York University and California University. The competition is held between the same number of students from both universities. The university that has more excellent students wins the competition. If the two universities have the same number of excellent students, the competition ends in a draw.

An excellent student is a student that scored 90% or more in the exam.

Return:
- **"New York University"** if New York University wins the competition.
- **"California University"** if California University wins the competition.
- **"No Winner"** if the competition ends in a draw.

The result format is in the following example.

**Example 1:**

**Input:** 

**NewYork table:**

| student_id | score |
|------------|-------|
| 1          | 90    |
| 2          | 87    |

**California table:**
| student_id | score |
|------------|-------|
| 2          | 89    |
| 3          | 88    |

**Output:** 
| winner              |
|---------------------|
| New York University |

**Explanation:**
New York University has 1 excellent student, and California University has 0 excellent students.


In [0]:
newyork_data_2072 = [(1, 90), (2, 87)]

newyork_columns_2072 = ["student_id", "score"]
newyork_df_2072 = spark.createDataFrame(newyork_data_2072, newyork_columns_2072)
newyork_df_2072.show()

california_data_2072 = [(2, 89), (3, 88)]

california_columns_2072 = ["student_id", "score"]
california_df_2072 = spark.createDataFrame(california_data_2072, california_columns_2072)
california_df_2072.show()

+----------+-----+
|student_id|score|
+----------+-----+
|         1|   90|
|         2|   87|
+----------+-----+

+----------+-----+
|student_id|score|
+----------+-----+
|         2|   89|
|         3|   88|
+----------+-----+



In [0]:
ny_df_2072 = newyork_df_2072.\
                        filter(col("score") >= 90).agg(count("*").alias("excellent")) \
                            .withColumn("university", lit("New York University"))

In [0]:
ca_df_2072 = california_df_2072\
                        .filter(col("score") >= 90).agg(count("*").alias("excellent")) \
                            .withColumn("university", lit("California University"))

In [0]:
df_2072 = ny_df_2072.union(ca_df_2072)

In [0]:
windowSpec = Window.orderBy(col("excellent").desc())

In [0]:
ranked_df_2072 = df_2072\
                    .withColumn("rnk", 
                                when(col("excellent") == df_2072.agg({"excellent":"max"}).collect()[0][0], 1)
                                .otherwise(2)
                                )

In [0]:
winner_df_2072 = ranked_df_2072\
                    .filter(col("rnk") == 1)

In [0]:
winner_df_2072\
    .groupBy().agg(
        when(count("*") > 1, lit("No Winner"))
        .otherwise(winner_df_2072.first()["university"]).alias("winner")
    ).show()

+-------------------+
|             winner|
+-------------------+
|New York University|
+-------------------+

